## Challenge: Exercise 4

In [1]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### To match searches with bookings

In [2]:
import bz2
files_path = "/home/dsc/Repositories/Master-in-Data-Science/Data/"

##### Loading and formatting bookings

In [5]:
bookings_file = bz2.BZ2File(files_path+"bookings.csv.bz2")
bookings_df = pd.read_csv(bookings_file, 
                          sep='^',
                          nrows=1e5,
                          usecols=['dep_port','arr_port','brd_time           ','pax'], 
                          parse_dates=['brd_time           '])

In [6]:
# Discarding "negative" bookings
bookings_df = bookings_df[bookings_df['pax'] >=1]
bookings_df.drop('pax', axis=1, inplace=True)

# Formatting bookings dataframe to match searches dataframe
bookings_df['Origin'] = bookings_df['dep_port'].apply(lambda s: s.strip())
bookings_df.drop('dep_port', axis=1, inplace=True)

bookings_df['Destination'] = bookings_df['arr_port'].apply(lambda s: s.strip())
bookings_df.drop('arr_port', axis=1, inplace=True)

bookings_df['Seg1Date'] = bookings_df['brd_time           '].apply(lambda d: pd.to_datetime(d.date()))
bookings_df.drop('brd_time           ', axis=1, inplace=True)

# Adding requested column "if search = booking"
bookings_df['Is_Searched'] = 1



##### Loading and formatting searches

In [7]:
searches_file = bz2.BZ2File(files_path+"searches.csv.bz2")
searches_df = pd.read_csv(searches_file, 
                          sep='^',
                          nrows=1e5,
                          usecols=['Origin','Destination','Seg1Date'],
                          parse_dates=['Seg1Date'])
searches_df.reset_index(inplace=True)

##### Merging both dataframes

In [8]:
# Obtaining searches with bookings
merged1_df = pd.merge(searches_df,bookings_df,how="inner").drop_duplicates()

In [9]:
# Obtaining searches without bookings
merged0_df = pd.merge(searches_df,bookings_df,how="outer")
merged0_df['Is_Searched'].fillna(0, inplace=True)
merged0_df = merged0_df[merged0_df['Is_Searched'] == 0]

In [15]:
# Appending and sorting final result
sol_df = merged1_df.append(merged0_df,ignore_index=True)
sol_df = sol_df.sort('index')
sol_df.head(10)

,index,Origin,Destination,Seg1Date,Is_Searched
1364,0,TXL,AUH,2013-01-26,0
1365,1,ATH,MIL,2013-01-04,0
1366,2,ICT,SFO,2013-08-02,0
1367,3,RNB,ARN,2013-01-02,0
1368,4,OSL,MAD,2013-03-22,0
1369,5,IAH,BLR,2013-02-14,0
1370,6,CPH,PAR,2013-03-02,0
1371,7,PAR,DUB,2013-02-12,0
1372,8,DUS,ACE,2013-03-30,0
1373,9,FRA,BGW,2013-02-26,0


In [14]:
sol_df.groupby('Is_Searched').count()

,index,Origin,Destination,Seg1Date
Is_Searched,,,,
0,98636,98636,98636,98241
1,1364,1364,1364,1364
